In [196]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import backend as K

In [377]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, LearningRateScheduler

In [62]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 2s 0us/step


In [64]:
X_train = X_train/255.0
X_test = X_test/255.0

In [345]:
init = tf.keras.initializers.RandomNormal(
        mean=0.0, stddev=0.005, seed=None
)

In [378]:
class layer(Model):
    def __init__(self, L:list, final_dim = 10):
        super().__init__()
        self.dense_ = [Dense(i, activation = "relu") for i in L]
        self.drop = [Dropout(0.2) for _ in self.dense_]
        self.final = Dense(final_dim, "softmax", kernel_initializer= init)
        self.normalize = [BatchNormalization() for _ in self.dense_]
    def call(self, x, train = None):
        x = Flatten()(x)
        for lay, drop, norm in zip(self.dense_, self.drop, self.normalize):
            x = norm(x, train)
            x = drop(x, train)
            x = lay(x)            
        return self.final(x)

In [435]:
tf.keras.utils.set_random_seed(10)
model = layer([128, 100, 50, 25], 10)

In [436]:
def scheduler(epoch, lr):
  if epoch < 2:
    return lr
  else:
    return lr * tf.math.exp(-0.1)


In [437]:
callback = LearningRateScheduler(scheduler) #### Seee the techical word cosine schedule 
callback_ = CSVLogger("log.csv")

In [438]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

In [439]:
model.fit(X_train, y_train, batch_size = 32, epochs = 1, validation_data= (X_test, y_test), callbacks = [callback, callback_])

1875/1875 [==============================] - 16s 8ms/step - loss: 0.6734 - accuracy: 0.7554 - val_loss: 0.4331 - val_accuracy: 0.8450 - lr: 0.0010


In [ ]:
model(np.random.randn(1,28,28))  ### you need to pass a fake data to load the weights
callback = ModelCheckpoint("model.hdf5", save_weights_only = True)
model.load_weights("model.hdf5") 